In [397]:
import pandas as pd
import string
import openpyxl
import datetime
import random
import os

In [398]:

# Assuming 'User_database.xlsx' is in the same directory as the notebook/script
excel_file1 = '/kaggle/input/d/mugdhazadkar/user-database/User_database.xlsx'
df_user = pd.read_excel(excel_file1)

# Load key_database 
key_database_path ='/kaggle/input/key-database/key_database.xlsx'
key_database = openpyxl.load_workbook(key_database_path)
worksheet = key_database.active

# Define the headers for the key database if it's empty
if worksheet.max_row == 0:
    headers = ["key_label", "key_value", "user_id", "session_id", "key_length", "key_domain", "generation_datetime"]
    worksheet.append(headers)

In [399]:
#user_authentication function

def authenticate_user(username, password):
    user_match = df_user[(df_user['username'] == username) & (df_user['password'] == password)]
    
    if not user_match.empty:
        return 'User authenticated'
    else:
        return 'User not recognized'

# Get user input
username = input("Enter your username: ")
password = input("Enter your password: ")

# Authenticate the user
result = authenticate_user(username, password)
print(result)

Enter your username:  kl;nm
Enter your password:  njm


User not recognized


In [400]:
# replacement for Gokul's function
def random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

# Function to convert bytes to hexadecimal
def bytes_to_hex(data):
    return ''.join(['{:02x}'.format(byte) for byte in data])

#key generation function
def key_generation(user_id, session_id, key_length, key_domain):
    generation_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key_value = random_string(key_length)
    key_label = random.randint(10000, 99999)  # Generate a key label
    key_value_hex = bytes_to_hex(key_value.encode('utf-8'))
    
    # Create a new workbook with a visible sheet
    new_key_database = openpyxl.Workbook()
    new_worksheet = new_key_database.active

    # Add headers to the new worksheet if it's empty
    if new_worksheet.max_row == 0:
        headers = ["key_label", "key_value", "user_id", "session_id", "key_length", "key_domain", "generation_datetime"]
        new_worksheet.append(headers)
    
    new_key = [key_label, key_value_hex, user_id, session_id, key_length, key_domain, generation_datetime]
    new_worksheet.append(new_key)
    
    # Save the new workbook
    new_key_database_path = '/kaggle/working/new_key_database.xlsx'
    new_key_database.save(new_key_database_path)
    
    # Load the existing workbook
    existing_key_database_path = '/kaggle/working/key_database.xlsx'
    existing_key_database = openpyxl.load_workbook(existing_key_database_path)
    
    # Access the default worksheet (the first sheet)
    existing_worksheet = existing_key_database.active
    
    # Append the new key data to the existing worksheet
    existing_worksheet.append(new_key)
    
    # Save both workbooks
    existing_key_database.save(existing_key_database_path)
    print("Keys saved to both new_key_database.xlsx and key_database.xlsx.")


In [401]:
# Function to delete a key from the key database
def key_termination(key_label):
     try:
        # Find the row with the specified key_label
        for row in worksheet.iter_rows(min_row=2, values_only=True):
            if row[0] == key_label:
                worksheet.delete_rows(row[0].row)
                key_database.save(key_database_path)
                print(f"Deleted rows with key_label '{key_label}'.")
                return
        print(f"Key label '{key_label}' not found in the database.")
     except Exception as e:
        print(f"An error occurred: {str(e)}")

In [406]:
# Example usage of the key_generation function
user_id = "mugdha"
session_id = 101
key_length = 16
key_domain = "QURE-derived"


key_generation(user_id, session_id, key_length, key_domain)
df1 = pd.read_excel("key_database.xlsx")
print(df1)

Keys saved to both new_key_database.xlsx and key_database.xlsx.
   13099  5345594667435938417a656767415558  mugdha  101  16  QURE-derived  \
0  51909  7370584c556c4d707a79376457534546  mugdha  101  16  QURE-derived   
1  30796  7141367733666d79714f556f52773153  mugdha  101  16  QURE-derived   
2  89033  506573594151583845434c34716f6d54  mugdha  101  16  QURE-derived   
3  77272  52617063754d514563677653665a6a49  mugdha  101  16  QURE-derived   
4  88753  4e7561584b5845367a6b36337641795a  mugdha  101  16  QURE-derived   

   2023-09-21 12:44:49  
0  2023-09-21 12:52:35  
1  2023-09-21 12:52:53  
2  2023-09-21 12:53:21  
3  2023-09-21 12:54:06  
4  2023-09-21 12:54:25  


In [407]:
delete_key = 81457
key_termination(delete_key)

#Load the updated workbook
df2=pd.read_excel("key_database.xlsx")
print(df2)

Key label '81457' not found in the database.
   13099  5345594667435938417a656767415558  mugdha  101  16  QURE-derived  \
0  51909  7370584c556c4d707a79376457534546  mugdha  101  16  QURE-derived   
1  30796  7141367733666d79714f556f52773153  mugdha  101  16  QURE-derived   
2  89033  506573594151583845434c34716f6d54  mugdha  101  16  QURE-derived   
3  77272  52617063754d514563677653665a6a49  mugdha  101  16  QURE-derived   
4  88753  4e7561584b5845367a6b36337641795a  mugdha  101  16  QURE-derived   

   2023-09-21 12:44:49  
0  2023-09-21 12:52:35  
1  2023-09-21 12:52:53  
2  2023-09-21 12:53:21  
3  2023-09-21 12:54:06  
4  2023-09-21 12:54:25  
